# MobileNetV2

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras import preprocessing
from tensorflow import keras
from keras import layers
from keras import regularizers
import pandas as pd

import os

from tensorflow import keras

import tensorflow_addons as tfa

from datetime import datetime

from hyperas import optim
from hyperopt import Trials, tpe

import cv2
from keras_preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix

In [3]:
test_path = "/Users/alexk/Documents/GitHub/Deep_Learning_for_Binary_Classification_of_Infectious_Keratitis/data"
x_test = []
# Put all hyperparameters in a dictionary
hyperparameters = {'batch_size': 128,
                   'random_flip': 'horizontal_and_vertical',
                   'random_rotation': (0.2),
                   'dropout': 0.5,
                   'L2': 0.3,
                   'base_LR': 0.001,
                   'initial_epochs': 20,
                   'fine_tune_epochs': 50,
                   'frozen_layer': 72}

# Declare all necessary variables
BATCH_SIZE = hyperparameters['batch_size']
IMG_SIZE = (224, 224)
path = "/Users/alexk/Documents/GitHub/Deep_Learning_for_Binary_Classification_of_Infectious_Keratitis/data"

In [4]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

for folder in listdir_nohidden(test_path):

    sub_path=test_path+"/"+folder

    for img in listdir_nohidden(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

In [ ]:
#predicted_classes = np.argmax(mlpmixer_classifier.predict(X, axis = 1))
test_datagen = ImageDataGenerator(rescale = 1./255)

test_x=np.array(x_test)

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_y=test_set.classes

print(test_y.shape)

In [5]:
# Preprocess images to the standard for a MobileNetV2
train_dataset = keras.preprocessing.image_dataset_from_directory(path,
                                                                 shuffle=True,
                                                                 subset='training',
                                                                 seed=42,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE,
                                                                 validation_split=0.2,
                                                                 label_mode='categorical',
                                                                 class_names=['Bacterial', 'Fungal'])
validation_dataset = keras.preprocessing.image_dataset_from_directory(path,
                                                                      shuffle=True,
                                                                      subset='validation',
                                                                      seed=42,
                                                                      batch_size=BATCH_SIZE,
                                                                      validation_split=0.2,
                                                                      image_size=IMG_SIZE,
                                                                      label_mode='categorical',
                                                                      class_names=['Bacterial', 'Fungal'])

Found 671 files belonging to 2 classes.
Using 537 files for training.
Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB

Found 671 files belonging to 2 classes.
Using 134 files for validation.


In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    def data_augmenter():
        """
            Create a sequential model composed of horizontal flips and random contrast adjustments
        """
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip(hyperparameters['random_flip']),
            tf.keras.layers.RandomRotation(factor=hyperparameters['random_rotation'])])
        return data_augmentation

data_augmentation = data_augmenter()

IMG_SHAPE = IMG_SIZE + (3,)
conv_base = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights='imagenet')


def MobileNetUlcerModel(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    # freeze the convolutional base
    conv_base.trainable = False

    # create the input layer
    inputs = tf.keras.Input(shape=IMG_SHAPE)

    # apply data augmentation to the inputs
    x = data_augmentation(inputs)

    # data preprocessing using the same weights as the original pre-trained model
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = conv_base(x, training=False)

    # Add the new binary classification layers
    # global average pooling layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # include dropout for regularization effect
    x = tf.keras.layers.Dropout(hyperparameters['dropout'])(x)

    # Add binary prediction layer
    outputs = tf.keras.layers.Dense(2, activation='softmax', kernel_regularizer=regularizers.l2(hyperparameters['L2']))(x)

    # Add sigmoid output layer if necessary (use AUC for classification metric of prediction)

    model = tf.keras.Model(inputs, outputs)

    return model

model = MobileNetUlcerModel()

base_learning_rate = hyperparameters['base_LR']

conv_base = model.layers[4]

conv_base.trainable = True

for layer in conv_base.layers[:hyperparameters['frozen_layer']]:
    layer.trainable = False

#model.summary()
# Compile the mobilenet using an RMSprop optimizer (gradient descent based optimizer)
#model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1 * hyperparameters['base_LR']),
#              loss='binary_crossentropy', # measuring the loss using the binary cross-entropy function
#              metrics=['accuracy', 'AUC']) # metrics of accuracy are AUC and accuracy

#with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
#    history = model.fit(train_dataset,
#                        epochs=hyperparameters['fine_tune_epochs'],
#                        validation_data=validation_dataset)

#model.summary()
# SGD based optimizer for MobileNetV2 (Gradient descent optimization)
#model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1 * hyperparameters['base_LR']),
#              loss='binary_crossentropy',
#              metrics=['accuracy', 'AUC'])

#with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
#    history_sgd = model.fit(train_dataset,
#                            validation_data=validation_dataset,
#                            epochs=hyperparameters['fine_tune_epochs'])

model.summary()
#Base learning rate Adam optimized MobileNetV2
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1 * hyperparameters['base_LR']),
                             loss='categorical_crossentropy',
                             metrics=['accuracy', 'AUC'])

with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    history_adam = model.fit(train_dataset,
                         validation_data=validation_dataset,
                         epochs=30)


# Achieves a 0.9552 peak validation accuracy indicating that MobileNet can handle data augmentation
model.summary()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1 * hyperparameters['base_LR']),
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    history_tuning = model.fit(train_dataset,
                            epochs=hyperparameters['fine_tune_epochs'],
                            validation_data=validation_dataset,
                               callbacks=[early_stopping])

df_loss_acc = pd.DataFrame(history_tuning.history)
df_loss = df_loss_acc[['loss', 'val_loss']]
df_loss.rename(columns={'loss': 'train', 'val_loss': 'validation'}, inplace=True)
df_acc = df_loss_acc[['accuracy', 'val_accuracy']]
df_acc.rename(columns={'accuracy': 'train', 'val_accuracy': 'validation'}, inplace=True)
df_auc = df_loss_acc[['auc', 'val_auc']]
df_auc.rename(columns={'auc': 'train', 'val_auc': 'validation'}, inplace=True)
df_loss.plot(title='Model loss', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Loss')
df_auc.plot(title='Model AUC', figsize=(12, 8)).set(xlabel='Epoch', ylabel='AUC')
df_acc.plot(title='Model Accuracy', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Accuracy')
plt.show()

#model.save('/content/drive/MyDrive/Machine Learning Research Files/LeNet and MobileNetV2 For Binary Classification of Infectious Keratitis/LeNet-MobileNetV2-For-Binary-Classification-of-Infectious-Keratitis/models/MobileNet_whole_image_fine_tune')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                             

2023-04-21 21:38:52.553524: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


5/5 [==============================] - 21s 2s/step - loss: 2.4293 - accuracy: 0.5903 - auc: 0.6023 - val_loss: 1.8234 - val_accuracy: 0.6343 - val_auc: 0.6219
Epoch 2/30


KeyboardInterrupt: 

In [58]:
mobilenet_predicted_classes = np.argmax(model.predict(test_x), axis=1)
mobilenet_cm = confusion_matrix(test_y, mobilenet_predicted_classes)
print(mobilenet_cm)

mobilenet_TN = mobilenet_cm[0][0]
mobilenet_FN = mobilenet_cm[0][1]
mobilenet_FP = mobilenet_cm[1][0]
mobilenet_TP = mobilenet_cm[1][1]
mobilenet_sensitivity = mobilenet_TP / (mobilenet_TP + mobilenet_FN)
mobilenet_specificity = mobilenet_TN / (mobilenet_TN + mobilenet_FP)

mobilenet_accuracy = (mobilenet_TP + mobilenet_TN) / 671

print(mobilenet_sensitivity)
print(mobilenet_specificity)
print(mobilenet_accuracy)

[[174  57]
 [ 54 386]]
0.871331828442438
0.7631578947368421
0.834575260804769
